# Analysis of a player's FACEIT data
## Imports & Environment Variables

In [5]:
import requests
from datetime import datetime, timedelta, timezone
from dotenv import load_dotenv
import pandas as pd
import os

load_dotenv()
bearer_token = os.getenv("FACEIT_API_BEARER_TOKEN")
headers = {
    "Authorization": f"Bearer {bearer_token}"
}

url_base = "https://open.faceit.com/data/v4"
nickname = "donk666"

from_datetime = datetime(2025, 1, 1).astimezone(timezone.utc)
# to_datetime = datetime(2025, 7, 16).astimezone(timezone.utc)
to_datetime = datetime.now(timezone.utc)

## Retrieve Player Identifier

In [6]:
params = {
    "nickname": nickname
}
response = requests.get(f"{url_base}/players", headers=headers, params=params)

if response.status_code != 200:
    raise Exception(f"Failed to retrieve data: {response.status_code}")

player_id = response.json().get("player_id", "")

## Retrieve Match Data

In [7]:
url = f"{url_base}/players/{player_id}/games/cs2/stats"

from_millis = int(from_datetime.timestamp() * 1000)
to_millis = int(to_datetime.timestamp() * 1000)

all_items = []
offset = 0
limit = 100

current_from_millis = from_millis

while current_from_millis < to_millis:
    next_month = datetime.fromtimestamp(current_from_millis / 1000).replace(day=28) + timedelta(days=4)
    next_month = next_month.replace(day=1)
    current_to_millis = int(next_month.timestamp() * 1000)

    params = {
        "from": str(current_from_millis),
        "to": str(min(current_to_millis, to_millis)),
        "limit": str(limit),
        "offset": str(offset)
    }

    while True:
        params["offset"] = str(offset)
        response = requests.get(url, params=params, headers=headers)

        if response.status_code != 200:
            print(f"Failed to retrieve data: {response.status_code}")
            break

        data = response.json()
        items = data.get("items", [])

        if not items:
            break

        all_items.extend(items)
        offset += limit

    current_from_millis = current_to_millis
    offset = 0

## Sort & Display Data

In [8]:
table_data = []
for item in all_items:
    stats = item.get("stats", {})
    kd_ratio = float(stats.get("K/D Ratio", float('inf')))
    kills = int(stats.get("Kills", 0))
    deaths = int(stats.get("Deaths", 0))
    score = stats.get("Score", "")
    map_name = stats.get("Map", "")
    match_finished_at = stats.get("Match Finished At", "")
    adr = float(stats.get("ADR", 0))
    
    match_finished_at = datetime.fromtimestamp(int(match_finished_at) / 1000).strftime('%Y-%m-%d %H:%M:%S')
    match_data = {
        "Kills": kills,
        "Deaths": deaths,
        "K/D Ratio": kd_ratio,
        "Score": score,
        "Map": map_name,
        "Match Finished At": match_finished_at,
        "ADR": adr,
        "Match Id": stats.get("Match Id")
    }
    
    table_data.append(match_data)

df = pd.DataFrame(table_data)
df = df.sort_values(by="Kills", ascending=True).head(20)

df

,Kills,Deaths,K/D Ratio,Score,Map,Match Finished At,ADR,Match Id
274,8,16,0.50,3 / 13,de_ancient,2025-04-23 12:39:35,71.6,1-0b3cf6ea-34e1-40d9-89ca-8203dfc67e52
107,8,15,0.53,13 / 3,de_mirage,2025-02-28 16:01:38,65.4,1-e92710cc-dd14-47c1-b7ab-29a2899162e8
66,9,19,0.47,9 / 13,de_mirage,2025-01-13 15:44:01,83.4,1-89936896-0fda-4634-a4db-137894db1186
395,9,16,0.56,13 / 4,de_anubis,2025-05-21 15:37:17,63.1,1-28d95d35-898e-4260-9f00-c31837fd2652
17,9,16,0.56,13 / 4,de_mirage,2025-01-27 14:24:29,66.3,1-e143fe2d-bc72-43d6-b0dd-94b28153bfa7
394,9,16,0.56,13 / 5,de_ancient,2025-05-21 16:32:44,67.2,1-7e23a48d-398b-443e-8fbd-f6c8713c4199
228,10,14,0.71,13 / 1,de_mirage,2025-03-04 14:22:54,61.9,1-92977be1-8968-4b0a-866d-ca8ce01bd393
364,11,14,0.79,13 / 3,de_mirage,2025-05-31 13:46:18,69.6,1-39594055-d3c7-4314-8bc3-eabd0715dd9f
528,11,17,0.65,13 / 9,de_anubis,2025-07-11 05:48:50,53.4,1-e1a367ba-8dbb-4696-acc5-8dbea564c8db
301,11,15,0.73,13 / 4,de_ancient,2025-04-16 15:49:01,75.7,1-a10b3720-b745-46a1-8851-32fd095767ce


# Match Duration Analysis

In [9]:
url = f"{url_base}/players/{player_id}/history"

all_items = []
offset = 0
limit = 100

current_from_datetime = from_datetime

while current_from_datetime < to_datetime:
    next_month = current_from_datetime.replace(day=28) + timedelta(days=4)
    next_month = next_month.replace(day=1)
    current_to_datetime = min(next_month, to_datetime)

    params = {
        "from": str(int(current_from_datetime.timestamp())),
        "to": str(int(current_to_datetime.timestamp())),
        "limit": str(limit),
        "offset": str(offset)
    }

    while True:
        params["offset"] = str(offset)
        response = requests.get(url, params=params, headers=headers)

        if response.status_code != 200:
            print(f"Failed to retrieve data: {response.status_code}")
            break

        items = response.json().get("items", [])

        if not items:
            break

        all_items.extend(items)
        offset += limit

    current_from_datetime = current_to_datetime
    offset = 0

In [10]:
average_duration = 0

for item in all_items:
    average_duration += item["finished_at"] - item["started_at"]

average_duration /= len(all_items)
print(f"Average Match Duration ({len(all_items)} matches): {str(timedelta(seconds=average_duration))}")
print(f"Maximum Match Duration: {str(timedelta(seconds=max(item['finished_at'] - item['started_at'] for item in all_items)))}")

Average Match Duration (543 matches): 0:34:53.491713
Maximum Match Duration: 1:41:31
